In [1]:
import os
import pandas as pd
import sys
os.chdir('/home/azureuser/cloudfiles/code/Users/DRumbleADMUK/lor-gdp-tools')

In [2]:
from gdp_tools.lor_gdp_tools import GlobalDataPlatformTools
from dotenv import load_dotenv
load_dotenv()
USERNAME = os.getenv('USERNAME')
PWD = os.getenv('PASSWORD')
SERVER = os.getenv('PROD_SERVER')
DATABASE = os.getenv('PROD_DATABASE') 
#SERVER = os.getenv('PREPROD_SERVER')
#DATABASE = os.getenv('PREPROD_DATABASE') 

# Create an instance of the clas
gdp_tools = GlobalDataPlatformTools(SERVER,DATABASE,USERNAME,PWD)

using credentials of drumble_adm_sql
ODBC Driver 18 for SQL Server is already installed.
Connected Successfully


In [6]:
# Call the query_gdp_to_pd function
sql_query = '''
            select top 10 * from [BASE].[Salesforce_CM_Client_Interview_summary__c] 
            where Comments_Long__c is not null
            '''
result = gdp_tools.query_gdp_to_pd(sql_query)
result.head()

/mnt/batch/tasks/shared/LS_root/mounts/clusters/drumbleadmuk1/code/Users/DRumbleADMUK/lor-gdp-tools/gdp_tools/lor_gdp_tools.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql_query, self.conn)


,SnapshotDate,TimeSlice,DataPlatformRowId,Business_Unit__c,Comments_Long__c,Comments_Short__c,Earliest_Project_Feedback__c,Earliest_Project_Feedback_Date__c,Last_Interview_Type__c,Last_Interviewee__c,...,Project_Director__c,Project_Leader__c,Sector__c,Start_On_Site_Date__c,LastModifiedDate,Id,IsDeleted,Interview_being_scheduled_for__c,LoadId,LoadDate
0,2024-09-27,2024-09-27,1290599,Building,xxxx,xxxx,a0W3z00000dVocTEAS,2022-12-05,2 - Delivery,xxxx,...,0032000000RjzMsAAJ,0033z00003BfWIZAA3,Sport & Leisure,2021-11-22,2024-09-09 12:46:25,a1P3z00000FwvkpEAB,False,NaT,4446061,2024-09-28 07:17:22.973
1,2024-08-26,2024-08-26,1126298,Building,xxxx,xxxx,a0WN20000022o7dMAA,2024-07-30,PCSA/ Pre-Delivery,xxxx,...,003D000002HtCpxIAF,None,Healthcare,2025-10-21,2024-08-15 20:12:50,a1P3z00000Fwvm2EAB,False,2024-05-08,3697562,2024-08-27 07:14:42.347
2,2024-10-10,2024-10-10,1356949,Building,xxxx,xxxx,a0W3z00000V6Yj0EAF,2021-06-09,2 - Delivery,xxxx,...,None,None,Sport & Leisure,2021-03-01,2024-09-09 12:46:10,a1P3z00000Fwvl0EAB,False,NaT,4678626,2024-10-11 07:22:59.460
3,2024-02-18,2024-02-18,165736,Building,xxxx,xxxx,a0W3z00000icqiEEAQ,2023-03-30,3 - Practical Completion,xxxx,...,None,0032000000RjxcMAAR,Healthcare,2018-11-05,2023-06-15 09:28:06,a1P3z00000Fwvl7EAB,False,NaT,1454475,2024-02-19 07:10:29.783
4,2024-04-25,2024-04-25,501819,Building,xxxx,xxxx,a0W3z00000W87mAEAR,2021-09-29,2 - Delivery,xxxx,...,None,003D000001y3PBjIAM,Sport & Leisure,2021-05-03,2024-03-08 16:17:15,a1P3z00000FwvlDEAR,False,NaT,1705036,2024-04-26 08:39:01.363


### Dev - saving data

In [4]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
#from azureml.core import Workspace, Dataset
#from azureml.data.datapath import DataPath
# Path is useful for almost all of AML as it can even assist
# references to paths that exist in the web
from pathlib import Path

In [7]:
# Azure credentials, this will automatically work in a compute
credential = DefaultAzureCredential()
user_name = 'DRumble'
project = 'example'

In [6]:
# convert to csv 
local_filename = 'output2.csv'
local_filepath = Path(f'data/raw/{local_filename}')

result.to_csv(local_filepath,index=False)

In [7]:
local_filename = 'output.csv'
local_filepath = Path(f'data/raw/{local_filename}')

blob_account_url = 'https://azstgdpdlpreproduks.blob.core.windows.net'
container = 'data-lake'
web_filename = f'LAB/{user_name}/{project}/data/raw/{local_filename}'


In [8]:
blob_client = (
    BlobServiceClient(account_url=blob_account_url, credential=credential)
    .get_container_client(container=container)
    .get_blob_client(blob=web_filename)
)

# Read in the local file to upload it
with open(local_filepath, 'rb') as data:
    blob_client.upload_blob(data, overwrite=True)

### Run from GDP Tools

In [7]:
user_name = 'EXample'
project = 'example'
local_filename = 'output.csv'
raw_data = True

gdp_tools.save_df_to_lab_storage(result,user_name,project,local_filename,raw_data)

data stored in lab location: LAB/EXample/example/data/raw/output.csv


### Loading data

In [8]:
user_name = 'EXample'
project = 'example'
local_filename = 'output.csv'
raw_data = True

df = gdp_tools.load_df_from_lab(user_name,project,local_filename,raw_data)

df.head()

https://azstgdpdlpreproduks.blob.core.windows.net/data-lake/LAB/EXample/example/data/raw/output.csv
Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration
Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


,SnapshotDate,TimeSlice,DataPlatformRowId,Business_Unit__c,Comments_Long__c,Comments_Short__c,Earliest_Project_Feedback__c,Earliest_Project_Feedback_Date__c,Last_Interview_Type__c,Last_Interviewee__c,...,Project_Director__c,Project_Leader__c,Sector__c,Start_On_Site_Date__c,LastModifiedDate,Id,IsDeleted,Interview_being_scheduled_for__c,LoadId,LoadDate
0,2024-09-27,2024-09-27,1290599,Building,xxxx,xxxx,a0W3z00000dVocTEAS,2022-12-05,2 - Delivery,xxxx,...,0032000000RjzMsAAJ,0033z00003BfWIZAA3,Sport & Leisure,2021-11-22,2024-09-09 12:46:25,a1P3z00000FwvkpEAB,False,NaT,4446061,2024-09-28 07:17:22.973
1,2024-08-26,2024-08-26,1126298,Building,xxxx,xxxx,a0WN20000022o7dMAA,2024-07-30,PCSA/ Pre-Delivery,xxxx,...,003D000002HtCpxIAF,None,Healthcare,2025-10-21,2024-08-15 20:12:50,a1P3z00000Fwvm2EAB,False,2024-05-08,3697562,2024-08-27 07:14:42.347
2,2024-10-10,2024-10-10,1356949,Building,xxxx,xxxx,a0W3z00000V6Yj0EAF,2021-06-09,2 - Delivery,xxxx,...,None,None,Sport & Leisure,2021-03-01,2024-09-09 12:46:10,a1P3z00000Fwvl0EAB,False,NaT,4678626,2024-10-11 07:22:59.460
3,2024-02-18,2024-02-18,165736,Building,xxxx,xxxx,a0W3z00000icqiEEAQ,2023-03-30,3 - Practical Completion,xxxx,...,None,0032000000RjxcMAAR,Healthcare,2018-11-05,2023-06-15 09:28:06,a1P3z00000Fwvl7EAB,False,NaT,1454475,2024-02-19 07:10:29.783
4,2024-04-25,2024-04-25,501819,Building,xxxx,xxxx,a0W3z00000W87mAEAR,2021-09-29,2 - Delivery,xxxx,...,None,003D000001y3PBjIAM,Sport & Leisure,2021-05-03,2024-03-08 16:17:15,a1P3z00000FwvlDEAR,False,NaT,1705036,2024-04-26 08:39:01.363


### Accessing data in sharepoint

### 

In [7]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
import pandas as pd
from io import StringIO
import getpass

In [5]:
# SharePoint site and credentials
site_url = 'https://laingorourke.sharepoint.com/:x:/s/DataandAnalytics-DataScienceandAnalytics/EVVzwvK-RPhKrsTQN39LbOwBiOubmwlmZvYVOsyBCMqMsw?e=aIJAPk'
username = getpass.getpass("Enter your Email: ")
password = getpass.getpass("Enter your password: ")

In [9]:
# Authentication
ctx_auth = AuthenticationContext(site_url)
if ctx_auth.acquire_token_for_user(username, password):
    ctx = ClientContext(site_url, ctx_auth)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    print("Authenticated into SharePoint site:", web.properties['Title'])
else:
    print("Authentication failed")

An error occurred while retrieving token from XML response: AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.


ValueError: An error occurred while retrieving token from XML response: AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.

In [ ]:

# Download the file
response = ctx.web.get_file_by_server_relative_url(file_url).download()
ctx.execute_query()

# Load the file content into a pandas DataFrame
file_content = StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_content)

In [ ]:
# not currently working

In [3]:
site_url = 'https://laingorourke.sharepoint.com/:x:/s/DataandAnalytics-DataScienceandAnalytics/EVVzwvK-RPhKrsTQN39LbOwBiOubmwlmZvYVOsyBCMqMsw?e=aIJAPk'

df = gdp_tools.load_df_from_sharepoint(site_url)
df.head()

An error occurred while retrieving token from XML response: AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.


ValueError: An error occurred while retrieving token from XML response: AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.